In [3]:
!pip install opencv-python
!pip install opencv-contrib-python
!pip install tensorflow
!pip install deepface
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=75885f8f70fb1255ea873a72f5a633ed7d38e3881754a57bae81a99518ada6cd
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 24.1 MB/s eta 0:00:00


In [4]:
# Configuração inicial para o Colab
from google.colab import drive
from google.colab.patches import cv2_imshow

# Montar o Google Drive para salvar o resultado (opcional)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace
import time
from tqdm import tqdm
import numpy as np

# Verificação da versão do OpenCV
print(f"Versão do OpenCV: {cv2.__version__}")

# Definição das emoções reconhecidas
emotions = {'angry': 0, 'disgust': 0, 'fear': 0, 'happy': 0, 'sad': 0, 'surprise': 0, 'neutral': 0}

Versão do OpenCV: 4.11.0


In [7]:
def detect_faces_with_rotation(frame, face_detector):
    """
    Detecta rostos em um frame, girando a imagem em múltiplos ângulos para lidar com rostos virados.

    Args:
        frame (numpy.ndarray): O frame de vídeo.
        face_detector: O objeto detector de rostos YuNet.

    Returns:
        list: Uma lista de rostos detectados (se houver), ou None se nenhum rosto for detectado.
    """
    detections = None

    for angle in [0, 90, 180, 270]:
        if angle != 0:
            (h, w) = frame.shape[:2]
            center = (w // 2, h // 2)
            M = cv2.getRotationMatrix2D(center, angle, 1.0)
            rotated = cv2.warpAffine(frame, M, (w, h))
        else:
            rotated = frame.copy()  # Crie uma cópia para evitar modificar o original

        face_detector.setInputSize((rotated.shape[1], rotated.shape[0]))
        _, faces = face_detector.detect(rotated)

        if faces is not None:
            # Ajustar as coordenadas das caixas delimitadoras para a imagem original
            if angle != 0:
                for face in faces:
                    x1, y1, w, h, confidence = map(int, face[:5])
                    # Desfazer a rotação das coordenadas
                    pts = np.array([[[x1, y1]], [[x1 + w, y1]], [[x1 + w, y1 + h]], [[x1, y1 + h]]], dtype=np.float32)
                    rotated_pts = cv2.transform(pts, M[:2])
                    x1_rotated = int(min(rotated_pts[0][0][0], rotated_pts[1][0][0], rotated_pts[2][0][0], rotated_pts[3][0][0]))
                    y1_rotated = int(min(rotated_pts[0][0][1], rotated_pts[1][0][1], rotated_pts[2][0][1], rotated_pts[3][0][1]))
                    w_rotated = int(max(rotated_pts[0][0][0], rotated_pts[1][0][0], rotated_pts[2][0][0], rotated_pts[3][0][0])) - x1_rotated
                    h_rotated = int(max(rotated_pts[0][0][1], rotated_pts[1][0][1], rotated_pts[2][0][1], rotated_pts[3][0][1])) - y1_rotated

                    face[:5] = [x1_rotated, y1_rotated, w_rotated, h_rotated, face[4]]  # Atualiza as coordenadas no array 'face'

            detections = faces
            break  # Pare de girar se encontrar algum rosto

    return detections

def process_video(video_path, output_path="output.mp4"):
    """
    Detecta rostos e emoções em um vídeo, utilizando YuNet para detecção de rostos e DeepFace para análise de emoções.

    Args:
        video_path (str): Caminho para o arquivo de vídeo de entrada.
        output_path (str, optional): Caminho para o arquivo de vídeo de saída com as detecções e emoções. Defaults to "output.avi".
    """
    # 1. Inicialização do Modelo YuNet para Detecção de Rostos
    modelPath = "/content/drive/MyDrive/Colab Notebooks/F4/face_detection_yunet_2023mar.onnx"
    face_detector = cv2.FaceDetectorYN_create(
        modelPath,
        "",  # Modelo de configuração (vazio para padrão)
        (640, 640),    # input_size (ajuste conforme necessário) - Changed to a tuple
        0.75,  # scoreThreshold (ajuste conforme necessário)
        0.1,  # nmsThreshold (ajuste conforme necessário)
        0 # Device ID (0 para CPU)
    )

    # 2. Configuração do Vídeo de Entrada e Saída
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print(f"Erro ao abrir o vídeo: {video_path}")
        return

    frame_width = int(video.get(3))
    frame_height = int(video.get(4))
    fps = int(video.get(cv2.CAP_PROP_FPS))
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # ou 'XVID' dependendo do codec disponível
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # 3. Loop Principal de Processamento do Vídeo
    frame_count = 0
    start_time = time.time()

    for _ in tqdm(range(total_frames), desc="Processando vídeo"):
        ret, frame = video.read()
        if not ret:
            break

        frame_count += 1

        # Define a resolução do detector de rostos
        face_detector.setInputSize((frame.shape[1], frame.shape[0]))

        # 4. Detecção de Rostos com YuNet
        faces = detect_faces_with_rotation(frame, face_detector)

        if faces is not None:
            # 5. Análise de Emoções com DeepFace para cada rosto detectado
            for face in faces:
                # Extrai as coordenadas do rosto
                x1, y1, w, h, confidence = map(int, face[:5])

                # Garante que as coordenadas estejam dentro dos limites da imagem
                x1 = max(0, x1)
                y1 = max(0, y1)
                w = min(w, frame.shape[1] - x1)
                h = min(h, frame.shape[0] - y1)

                # Extrai a região do rosto para análise de emoção
                face_img = frame[y1:y1+h, x1:x1+w]

                try:
                    # Analisa a emoção usando DeepFace
                    analysis = DeepFace.analyze(face_img, actions=['emotion'], enforce_detection=False, detector_backend='skip')
                    # print(f"Resultado da análise: {analysis}")  # Imprime o resultado completo
                    dominant_emotion = analysis[0]['dominant_emotion']

                    # Desenha a caixa delimitadora e a emoção no frame
                    cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
                    cv2.putText(frame, dominant_emotion, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    emotions[dominant_emotion] += 1

                except Exception as e:
                    # print(f"Erro ao analisar emoção: {e}")

                    print(f"Erro ao analisar emoção: {type(e).__name__}: {e}")  # Imprime o tipo e a mensagem de erro

                    cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)
                    cv2.putText(frame, "Erro", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


        # 6. Salva o frame processado no vídeo de saída
        out.write(frame)

    # 7. Liberação dos recursos
    video.release()
    out.release()
    cv2.destroyAllWindows()

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Vídeo processado: {output_path}")
    print(f"Tempo total de processamento: {elapsed_time:.2f} segundos")
    print(f"Número total de frames: {frame_count}")
    print(f"FPS médio: {frame_count / elapsed_time:.2f}")
    print(f"Emoções detectadas: {emotions}")
    for emotion, count in emotions.items():
        percentage = (count / frame_count) * 100
        print(f"{emotion}: {percentage:.2f}%")

In [11]:
# Caminhos para vídeo de entrada/saída
input_video_path = "/content/drive/MyDrive/Colab Notebooks/F4/input_shorter.mp4"  # Substituir pelo caminho do vídeo carregado
output_video_path = "/content/drive/MyDrive/Colab Notebooks/F4/output_shorter.mp4"


# Execute a função para analisar o vídeo
process_video(input_video_path, output_video_path)

Processando vídeo: 100%|██████████| 291/291 [00:37<00:00,  7.82it/s]

Vídeo processado: /content/drive/MyDrive/Colab Notebooks/F4/output_shorter.mp4
Tempo total de processamento: 37.21 segundos
Número total de frames: 291
FPS médio: 7.82
Emoções detectadas: {'angry': 186, 'disgust': 0, 'fear': 471, 'happy': 570, 'sad': 660, 'surprise': 9, 'neutral': 480}
Porcentagem de cada emoção: {'angry': 186, 'disgust': 0, 'fear': 471, 'happy': 570, 'sad': 660, 'surprise': 9, 'neutral': 480}
angry: 63.92%
disgust: 0.00%
fear: 161.86%
happy: 195.88%
sad: 226.80%
surprise: 3.09%
neutral: 164.95%


In [18]:
import mediapipe as mp
import torch
from mediapipe.python.solutions import pose as mp_pose
from tqdm import tqdm

yolo_model = torch.hub.load("ultralytics/yolov5", "yolov5s")

# since we are only intrested in detecting person
yolo_model.classes = [0]

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-2-15 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [13]:
def processar_video_multiplas_pessoas(input_video_path, output_video_path):

  cap = cv2.VideoCapture(input_video_path)

  frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

  out = cv2.VideoWriter(
      output_video_path,
      cv2.VideoWriter_fourcc(*"mp4v"),
      fps,
      (frame_width, frame_height),
  )

  for _ in tqdm(range(total_frames), desc="Processando vídeo"):
      ret, frame = cap.read()
      if not ret:
          break

      # Recolor Feed from RGB to BGR
      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      # making image writeable to false improves prediction
      image.flags.writeable = False

      result = yolo_model(image)

      # Recolor image back to BGR for rendering
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

      # This array will contain crops of images incase we need it
      img_list = []

      # we need some extra margin bounding box for human crops to be properly detected
      MARGIN = 10

      for xmin, ymin, xmax, ymax, confidence, clas in result.xyxy[0].tolist():
          with mp_pose.Pose(
              static_image_mode=False,
              model_complexity=2,
              smooth_landmarks=True,
              min_detection_confidence=0.4, min_tracking_confidence=0.4
          ) as pose:
              # Media pose prediction ,we are
              results = pose.process(
                  image[
                      int(ymin) + MARGIN : int(ymax) + MARGIN,
                      int(xmin) + MARGIN : int(xmax) + MARGIN :,
                  ]
              )

              # Draw landmarks on image, if this thing is confusing please consider going through numpy array slicing
              mp_drawing.draw_landmarks(
                  image[
                      int(ymin) + MARGIN : int(ymax) + MARGIN,
                      int(xmin) + MARGIN : int(xmax) + MARGIN :,
                  ],
                  results.pose_landmarks,
                  mp_pose.POSE_CONNECTIONS,
                  mp_drawing.DrawingSpec(
                      color=(245, 117, 66), thickness=2, circle_radius=2
                  ),
                  mp_drawing.DrawingSpec(
                      color=(245, 66, 230), thickness=2, circle_radius=2
                  ),
              )
              img_list.append(image[int(ymin) : int(ymax), int(xmin) : int(xmax) :])

      out.write(image)

  cap.release()
  out.release()
  cv2.destroyAllWindows()

In [14]:
# Caminhos para vídeo de entrada/saída
input_video_path = "/content/drive/MyDrive/Colab Notebooks/F4/input_end.mp4"  # Substituir pelo caminho do vídeo carregado
output_video_path = "/content/drive/MyDrive/Colab Notebooks/F4/output_end_poses.mp4"


# Execute a função para analisar o vídeo
processar_video_multiplas_pessoas(input_video_path, output_video_path)

Processando vídeo: 100%|██████████| 1576/1576 [17:13<00:00,  1.52it/s]


In [21]:
# Função para detectar se o braço está levantado
def detecta_braco_levantado(landmarks):
    """
    Detecta se a pessoa levantou o braço.

    Args:
        landmarks: Lista de landmarks do Mediapipe.

    Returns:
        True se o braço está levantado, False caso contrário.
    """
    ombro_direito = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    ombro_esquerdo = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    mao_direita = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    mao_esquerda = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

    # Define uma altura acima do ombro para considerar o braço levantado
    altura_acima_ombro = 0.1  # Ajuste este valor conforme necessário

    if mao_direita.y < ombro_direito.y - altura_acima_ombro or \
      mao_esquerda.y < ombro_esquerdo.y - altura_acima_ombro:
        return True
    else:
          return False

# Função para detectar giro do corpo (simplificada)
#  **IMPORTANTE:** Essa detecção de giro é muito simplificada e provavelmente
#  precisará de ajustes e refinamentos significativos para funcionar bem
#  em diferentes cenários.  Detectar um giro completo de forma robusta
#  é um problema complexo.
def detecta_giro(landmarks, historico_ombros, threshold=0.1):
    """
    Detecta se a pessoa girou o corpo.  **ESTA É UMA IMPLEMENTAÇÃO SIMPLIFICADA.**

    Args:
        landmarks: Lista de landmarks do Mediapipe.
        historico_ombros: Lista com as posições dos ombros em frames anteriores.
        threshold: Limiar para detectar a mudança na posição dos ombros.

    Returns:
        True se um giro for detectado, False caso contrário.
    """

    ombro_direito = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    ombro_esquerdo = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

    # Adiciona a posição atual dos ombros ao histórico
    historico_ombros.append(((ombro_direito.x, ombro_direito.y), (ombro_esquerdo.x, ombro_esquerdo.y)))

    # Mantém o histórico com um tamanho máximo (por exemplo, os últimos 10 frames)
    if len(historico_ombros) > 10:
        historico_ombros.pop(0)

    # Se não tivermos histórico suficiente, não podemos detectar o giro
    if len(historico_ombros) < 5: # Ajuste este valor conforme necessário
        return False

    # Calcula a diferença na posição dos ombros entre o frame atual e o frame anterior
    ombro_direito_anterior = historico_ombros[-2][0]
    ombro_esquerdo_anterior = historico_ombros[-2][1]

    diff_ombro_direito_x = ombro_direito.x - ombro_direito_anterior[0]
    diff_ombro_esquerdo_x = ombro_esquerdo.x - ombro_esquerdo_anterior[0]


    # Se a diferença na posição dos ombros for maior que um limiar, consideramos que houve um giro
    if abs(diff_ombro_direito_x) > threshold or abs(diff_ombro_esquerdo_x) > threshold:
        return True
    else:
        return False

In [22]:
# Caminhos para vídeo de entrada/saída
input_video_path = "/content/drive/MyDrive/Colab Notebooks/F4/input_end.mp4"  # Substituir pelo caminho do vídeo carregado
output_video_path = "/content/drive/MyDrive/Colab Notebooks/F4/output_end_poses.mp4"

# Inicialização do Mediapipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    enable_segmentation=False,
    smooth_segmentation=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)

# Carrega o vídeo
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Erro ao abrir o vídeo.")
    exit()

# Obtém informações do vídeo para criar o arquivo de saída
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define o codec e cria o objeto VideoWriter para salvar o vídeo
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # ou 'XVID'
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

historico_ombros = []  # Histórico para detecção de giro

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Converte a imagem para RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Processa a imagem com o Mediapipe
    results = pose.process(image)

    # Converte a imagem de volta para BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Desenha os landmarks na imagem (se detectados)
    if results.pose_landmarks:
        mp.drawing_utils.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Imprime os landmarks (opcional, pode ser muito verbose)
        # print("Landmarks:", results.pose_landmarks.landmark)

        # Detecta atividades
        landmarks = results.pose_landmarks.landmark
        if detecta_braco_levantado(landmarks):
            print("Atividade: Levantou o braço")
            cv2.putText(image, "Levantou o braco", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)  # Adiciona texto no vídeo

        if detecta_giro(landmarks, historico_ombros):
            print("Atividade: Girou o corpo")
            cv2.putText(image, "Girou o corpo", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)  # Adiciona texto no vídeo


    # Salva o frame no vídeo de saída
    out.write(image)

    # Sai do loop se a tecla 'q' for pressionada
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera os recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print("Vídeo processado e salvo em:", output_video_path)

AttributeError: module 'mediapipe' has no attribute 'drawing_utils'